# Topic modeling on local data (Kenya) using LatentDirichletAllocation from sklearn

## Import Libraries

In [ ]:
import pandas as pd
import re

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

## Import Data

In [ ]:
## Mount drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
twitter_data = pd.read_csv('/content/drive/MyDrive/Module 3/Datasets/Location Trend Tweets 2022-03-31.csv')
twitter_data

,screen_name,hashtag,tweet,time_stamp
0,EliasKabere,#BBIFinalVerdict,RT @Belive_Kinuthia: “IEBC was legally constit...,2022-03-31 08:47:01+00:00
1,Channel54News,#BBIFinalVerdict,"KENYA:#BBIFinalVerdict \n\n"" If the Supreme Co...",2022-03-31 08:47:00+00:00
2,KoneMoheavy,#BBIFinalVerdict,RT @BravinYuri: Summary of CJ Martha Koome's v...,2022-03-31 08:47:00+00:00
3,GodfearingDude,#BBIFinalVerdict,RT @ntvkenya: CJ Koome: I endorse the findings...,2022-03-31 08:46:59+00:00
4,godwin_sakaya,#BBIFinalVerdict,#Supreme court Judge William Ouko has acted th...,2022-03-31 08:46:59+00:00
...,...,...,...,...
2494,abdiazizhashim1,Mighty Diamonds,The BBI Susan Kihika Sonko Junet Odingas Ledam...,2022-03-31 08:30:00+00:00
2495,exclusiveska,Mighty Diamonds,RT @BigshipSounds: The Mighty Diamonds 🔥🔥 http...,2022-03-31 08:29:55+00:00
2496,Breasman1,Mighty Diamonds,RT @VPRecords: Devastated to hear of the passi...,2022-03-31 08:26:54+00:00
2497,royalrampnews,Mighty Diamonds,MIGHTY DIAMONDS Singer Shot &amp; Killed https...,2022-03-31 08:25:20+00:00


## Clean Data

In [ ]:
def text_cleaner (text):
  text = re.sub(r'@[A-Za-z0-9]+','',text) ## remove @ mentions
  text = re.sub(r'#','',text) ## remove # symbol
  text = re.sub(r'^RT+','',text) ## remove RT
  text = re.sub(r'https?:\/\/\S+','',text) ## remove hyperlink
  text = re.sub(r'[^\w\s]','',text) ## remove everything apart from words and space
  text = re.sub(r'_',' ',text) ## remove underscore
  text = re.sub(r'\n',' ',text) ## remove \n

  return text

In [ ]:
## Create clean text column

twitter_data['cleaned_tweet'] = twitter_data['tweet'].apply(text_cleaner)
twitter_data = twitter_data[['screen_name','hashtag','tweet','cleaned_tweet','time_stamp']]
twitter_data

,screen_name,hashtag,tweet,cleaned_tweet,time_stamp
0,EliasKabere,#BBIFinalVerdict,RT @Belive_Kinuthia: “IEBC was legally constit...,Kinuthia IEBC was legally constituted when i...,2022-03-31 08:47:01+00:00
1,Channel54News,#BBIFinalVerdict,"KENYA:#BBIFinalVerdict \n\n"" If the Supreme Co...",KENYABBIFinalVerdict If the Supreme Court r...,2022-03-31 08:47:00+00:00
2,KoneMoheavy,#BBIFinalVerdict,RT @BravinYuri: Summary of CJ Martha Koome's v...,Summary of CJ Martha Koomes verdict i On ba...,2022-03-31 08:47:00+00:00
3,GodfearingDude,#BBIFinalVerdict,RT @ntvkenya: CJ Koome: I endorse the findings...,CJ Koome I endorse the findings of the two s...,2022-03-31 08:46:59+00:00
4,godwin_sakaya,#BBIFinalVerdict,#Supreme court Judge William Ouko has acted th...,Supreme court Judge William Ouko has acted the...,2022-03-31 08:46:59+00:00
...,...,...,...,...,...
2494,abdiazizhashim1,Mighty Diamonds,The BBI Susan Kihika Sonko Junet Odingas Ledam...,The BBI Susan Kihika Sonko Junet Odingas Ledam...,2022-03-31 08:30:00+00:00
2495,exclusiveska,Mighty Diamonds,RT @BigshipSounds: The Mighty Diamonds 🔥🔥 http...,The Mighty Diamonds,2022-03-31 08:29:55+00:00
2496,Breasman1,Mighty Diamonds,RT @VPRecords: Devastated to hear of the passi...,Devastated to hear of the passing of Tabby D...,2022-03-31 08:26:54+00:00
2497,royalrampnews,Mighty Diamonds,MIGHTY DIAMONDS Singer Shot &amp; Killed https...,MIGHTY DIAMONDS Singer Shot amp Killed RIP Ta...,2022-03-31 08:25:20+00:00


In [ ]:
twitter_data['cleaned_tweet'][0]

'  Kinuthia IEBC was legally constituted when it undertook the verification of the Signatures for the BBI Amendment Bill  CJ M'

## Pre-Processing

In [ ]:
## create vocabulary

cv = CountVectorizer(max_df=0.9,min_df=5,stop_words='english')

In [ ]:
## Document term matrix
dtm = cv.fit_transform(twitter_data['cleaned_tweet'])
dtm

<2499x990 sparse matrix of type '<class 'numpy.int64'>'
	with 20513 stored elements in Compressed Sparse Row format>

## Modeling

In [ ]:
rand_topics = 3

from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=rand_topics,random_state=42)

LDA.fit(dtm)

LatentDirichletAllocation(n_components=3, random_state=42)

In [ ]:
## Grab vocabulay
print(len(cv.get_feature_names()))
cv.get_feature_names()[510:520]


990


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['kihika',
 'kileleshwa',
 'killed',
 'kindly',
 'kinuthia',
 'kinyanjui',
 'kiosk',
 'klopp',
 'knew',
 'know']

In [ ]:
## Grab Topics

print(LDA.components_.shape)

(3, 990)


In [ ]:
## Sample 1 topic

single_topic = LDA.components_[0]

## Order word distribution in topic 0 and get top ten words

topic_words = 30

top_words = single_topic.argsort()[-topic_words:]

for index in top_words:

  print(cv.get_feature_names()[index])

sports
na
april
mpesa
wanjiku
doctrine
fifa
networks
cameroon
kenya
algeria
uhuru
bring
nakuru
popular
bbifinalverdict
justice
like
structure
basic
match
constitution
initiative
supreme
president
martha
court
koome
cj
bbi


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
word_count = 15

for i,topic in enumerate(LDA.components_):
  print("The top  {word_count} word for topic # {i} is:".format(word_count=word_count,i=i))
  print([cv.get_feature_names()[index] for index in topic.argsort()[-word_count:]])
  print('\n')
  print('\n')

The top  15 word for topic # 0 is:
['bbifinalverdict', 'justice', 'like', 'structure', 'basic', 'match', 'constitution', 'initiative', 'supreme', 'president', 'martha', 'court', 'koome', 'cj', 'bbi']




The top  15 word for topic # 1 is:
['university', 'kileleshwa', 'uhuruto', 'odingas', 'diamonds', 'mighty', 'daily', 'sonko', 'kuria', 'nation', 'junet', 'bbi', 'kihika', 'susan', 'ledama']




The top  15 word for topic # 2 is:
['legally', 'junior', 'mendy', 'amendment', 'new', 'attorney', 'kcpe', 'people', 'iebc', 'bruce', 'willis', 'time', 'amp', 'hon', 'bbi']






/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
## Attach topics to original dataset
dtm

<2499x990 sparse matrix of type '<class 'numpy.int64'>'
	with 20513 stored elements in Compressed Sparse Row format>

In [ ]:
topic_results = LDA.transform(dtm)

topic_results.shape

(2499, 3)

In [ ]:
topic_results[0].round(2)

array([0.04, 0.03, 0.93])

In [ ]:
twitter_data['cleaned_tweet'][0]

'  Kinuthia IEBC was legally constituted when it undertook the verification of the Signatures for the BBI Amendment Bill  CJ M'

In [ ]:
twitter_data['topic'] = topic_results.argmax(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
twitter_data.head()

,screen_name,hashtag,tweet,cleaned_tweet,time_stamp,topic
0,EliasKabere,#BBIFinalVerdict,RT @Belive_Kinuthia: “IEBC was legally constit...,Kinuthia IEBC was legally constituted when i...,2022-03-31 08:47:01+00:00,2
1,Channel54News,#BBIFinalVerdict,"KENYA:#BBIFinalVerdict \n\n"" If the Supreme Co...",KENYABBIFinalVerdict If the Supreme Court r...,2022-03-31 08:47:00+00:00,0
2,KoneMoheavy,#BBIFinalVerdict,RT @BravinYuri: Summary of CJ Martha Koome's v...,Summary of CJ Martha Koomes verdict i On ba...,2022-03-31 08:47:00+00:00,0
3,GodfearingDude,#BBIFinalVerdict,RT @ntvkenya: CJ Koome: I endorse the findings...,CJ Koome I endorse the findings of the two s...,2022-03-31 08:46:59+00:00,0
4,godwin_sakaya,#BBIFinalVerdict,#Supreme court Judge William Ouko has acted th...,Supreme court Judge William Ouko has acted the...,2022-03-31 08:46:59+00:00,0


In [ ]:
twitter_data['topic'].value_counts(normalize=True)

0    0.383353
2    0.331333
1    0.285314
Name: topic, dtype: float64

## Visualizing the models with pyLDAvis

In [ ]:
pyLDAvis.sklearn.prepare(LDA, dtm, cv)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.324176 -0.003490       1        1  37.832792
0      0.165625 -0.238218       2        1  37.448087
2      0.158552  0.241708       3        1  24.719121, topic_info=          Term        Freq       Total Category  logprob  loglift
541     ledama  518.000000  518.000000  Default  30.0000  30.0000
870      susan  373.000000  373.000000  Default  29.0000  29.0000
510     kihika  362.000000  362.000000  Default  28.0000  28.0000
484      junet  303.000000  303.000000  Default  27.0000  27.0000
633     nation  273.000000  273.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
376    general   31.275833   49.623114   Topic3  -5.1238   0.9360
61   amendment   35.495730   65.601892   Topic3  -4.9972   0.7834
111        bbi   94.486393  625.507637   Topic3  -4.0182  -0.4925
235     county   26.658567   38.302426   Topic3  -5.2835   1.0352
646     nyando   28.994956   56.619651   Topic3  -5.1995   0.7284

[150 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         1  0.976850      072
9         3  0.947974      137
18        2  0.995577      2nd
44        3  0.956652    agnes
47        3  0.984267   airtel
...     ...       ...      ...
936       1  0.700551     void
936       2  0.287406     void
936       3  0.017963     void
945       2  0.981078  wanjiku
965       3  0.982298   willis

[154 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])